In [ ]:
# 데이터 증강 설정 (전달)
item_tfms = Resize(128, ResizeMethod.Squish)
batch_tfms = aug_transforms(
    do_flip=True,                # 이미지 좌우 반전
    flip_vert=True,              # 이미지 상하 반전
    max_rotate=30,               # 최대 30도까지 회전
    max_zoom=1.1,                # 최대 1.1배까지 확대
    max_lighting=0.2,            # 조명 변화
    max_warp=0.2,                # 왜곡 변화
    p_affine=0.75,               # 기하학적 변환 적용 확률
    p_lighting=0.75              # 조명 변환 적용 확률
)

# 추가적인 증강 기법 적용
batch_tfms += [RandomErasing(p=0.5, max_count=3)]  # 랜덤 지우기 기법 추가

# 데이터셋 로더 생성
typetools = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=item_tfms,
    batch_tfms=batch_tfms
)

dls = typetools.dataloaders(path)

# 모델 학습
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

### FastAi를 활용한 이미지분류

- 이번 실습에서는 Python FastAi 라이브러리를 활용하여
- Bing 검색 엔진에서 얻을 수 있는 이미지 주소를 통해
- 이미지 분류 딥러닝 모델을 학습시킬 데이터를 수집하고
- 이미지 분류 딥러닝 모델의 특성을 정의한 후
- 데이터 증강을 통해 이미지를 변형해보고
- 모델을 학습시킨 후 결과를 확인해본 후
- 실제 모델의 성능을 확인하는 실습을 진행해보겠습니다!

In [ ]:
#fastai2 라이브러리 설치
!pip install fastai2

In [ ]:
#한글 폰트 처리를 위한 라이브러리 설치
!pip install koreanize-matplotlib

In [ ]:
#fastbook 라이브러리 설치 및 불러오기
#실행이 안되면 다시 실행할 것~

!pip install -Uqq fastbook

In [ ]:
#코드가 드라이브에 접근할 수 있도록 허용해주는 코드
from google.colab import drive
drive.mount('/content/drive')

In [5]:
# fastbook 라이브러리에서 모든 도구들을 가져온다
# fastai.viwison.widgets에서 모든 도구들을 가져온다.
# 한글 폰트 처리를 위한 도구를 가져온다.

import fastbook
from fastbook import *
from fastai.vision.widgets import *
import koreanize_matplotlib

### 데이터 수집하기

* 항상 AI 모델 학습을 위해서는 AI 학습을 위한 데이터가 필요하다.
* MS의 검색엔진인 Bing을 이용하여
* 이미지를 자동으로 수집하고 이를 여러분의 구글 드라이브에 저장하는 과정을 진행해보자!
* 한글 키워드를 검색하여 다운도 가능하지만 추후 AI 모델 제작 과정에서 폰트가 깨지는 문제가 발생하기에.. 영어 키워드로 이미지를 수집해보자!
* 우리가 AI 모델을 만들어 활용하는 이유는 많은 문제가 있을 때 이를 빠르고 정확하게 해결하기 위해서이다. 이를 기억하고 실제 현실에서 활용할 수 있을 법한 문제를 해결하는 이미지를 수집해보자. (예 : 다양한 옷 이미지가 있을 때 이를 상의, 하의, 겉옷으로 분류해주는 문제)

In [ ]:
!pip install icrawler


In [ ]:
from icrawler.builtin import GoogleImageCrawler

# 저장 폴더 지정
google_crawler = GoogleImageCrawler(storage={'root_dir': 'images/연습'})
# 'cats' 키워드로 최대 200개 이미지 크롤링 및 저장
google_crawler.crawl(keyword='', max_num=10)


In [13]:
# 분류할 type을 key_types에 문자열로 저장한다.
key_types = '','',''
# 드라이브에 사진을 저장할 기본 폴더를 만든 후 이를 경로로 지정한다
path = Path('/content/drive/MyDrive/홍천고 이미지 분류')
# 만약 path에 저장된 경로가 없으면 경로를 만들어준다
if not path.exists() :
  path.mkdir()

In [ ]:
#o라는 이름으로 key_type을 돌면서
for o in key_types :
  print(path, o) # 확인용
  dest = (path/o) #각각의 키워드에 따라 사진을 저장할 폴더 경로
  print(dest) # 확인용
  dest.mkdir(exist_ok = True) #폴더가 없으면 폴더를 만들고
  google_crawler = GoogleImageCrawler(storage={'root_dir': dest})
  google_crawler.crawl(keyword=o, max_num=)


In [ ]:
#모든 사진파일 정보 확인
fns = get_image_files(path)
fns
#왜 300개가 아닐까?

In [ ]:
# 모든 사진파일의 경로가 들어있는 fns를 검증하여 실패한 사진의 경로만 따로 저장함
failed = verify_images(fns)
failed

In [ ]:
# 다운에 실패한 이밎의 위치를 Path에서 제거함
failed.map(Path.unlink)

### 데이터 전처리하기

* 이제 우리는 AI 학습을 위한 사진 파일 데이터 수집을 완료했다.
* 그런데 불러온 사진의 종류를 결정하는 데에는 이미지의 어떤 부분이 영향을 끼칠까?
* 또 학습할 때 꼭 많은 양의 이미지만 필요할까?
* 이미지의 다양한 특성을 모델이 학습할 수 있도록 이미지를 전처리할수는 없을까?

In [20]:
typetools = DataBlock(blocks = (ImageBlock, CategoryBlock), #독립변수(데이터), 종속변수(데이터의 정답)
                   get_items = get_image_files, #파일 경로 설정
                   splitter = RandomSplitter(valid_pct =0.2  , seed = 42 ), #검증 파일 비율 20% 랜덤, seed값 난수 고정(42), 20%는 공부하는데 쓰이는게 아니라 테스트하는데 사용됨. -> 본인이 생각하고 수정해도 됨
                   get_y = parent_label, #파일이 저장된 폴더명을 레이블로 사용
                   item_tfms = Resize(128)) #사이즈 통일

#typetools에는 학습을 위한 기본 조건이 들어간다. (공부법)

In [21]:
#공부법에 실제 데이터를 적용한다
#예) 50분 공부하고 10분 쉬고 암기로 공부하는 방법을 -> 영어단어에 적용한다.

dls = typetools.dataloaders(path) #공부방법.dataloaders(공부과목)

In [ ]:
#검증 데이터의 값 출력해서 확인하기
dls.valid.show_batch(max_n =8  ,nrows =2  ) #n개의 값을 n개의 줄에 나누어 출력한다.


In [ ]:
# 데이터 증강 1
#이미지 찌그러뜨리기
#이미지를 변형한 다른 이미지를 이용하여 학습한다면 이미지가 어떤 종류라는걸 나타내는데에 관여하는 특징을 좀 더 잘 파악할 수 있다.

typetools = typetools.new(item_tfms=Resize(128,ResizeMethod.Squish)) #Squish 찌그러뜨리다
dls = typetools.dataloaders(path)
dls.train.show_batch(max_n = 8  ,nrows = 2  )

In [ ]:
# 데이터 증강 2
#이미지 빈칸 채우기
typetools = typetools.new(item_tfms=Resize(128,ResizeMethod.Pad,pad_mode='zeros')) #빈칸을 검은색 zeros로 채움
dls = typetools.dataloaders(path)
dls.train.show_batch(max_n = 8 ,nrows = 2 )

### 이미지 영역 랜덤 설정하기

- 위에서 이미지를 찌그러뜨리거나 빈칸으로 채우는것은
- 학습하는데 있어 이미지를 왜곡하거나 필요없는 정보를 추가하는 문제가 생길 수 있다.
- 이에 대한 해결 방법으로
- 이미지를 바라보는 시점을 랜덤으로 설정한 후 학습하는 방법이 있다.
- 다만 그 전에 찌그러뜨리기 -> 빈칸 처리된 사진이 아니라 원본을 다시 불러오는 작업이 필요하다.

In [ ]:
#데이터 증강 3
typetools = typetools.new(item_tfms=RandomResizedCrop(128,min_scale=0.5)) # 128 크기에서 영역의 랜덤 이미지를 얼마나 지정할지 설정 min_scale 바꿔도 됨
dls = typetools.dataloaders(path)
dls.train.show_batch(max_n = 8, nrows=1, unique=True)  #유니크값 동일 이미지에 대한 영역 정보

### 이런 기법을 데이터 증강이라고 한다!

* 데이터 증강이란 입력 데이터를 임의로 변형해 새로운 데이터를 생성하는 기법을 의미한다.
* 회전, 뒤집기, 원근 뒤틀기, 명도 바꾸기, 채도 바꾸기 등이 존재하며
* 변형된 결과는 다양한 이미지를 만들지만 원본 데이터 자체의 이미를 바꿔서는 안된다.
* 데이터 증강을 사용하면 데이터에서 데이터의 종류를 결정짓는 특징을 좀 더 잘 추출할 수 있고
* 더 나아가 적은 양의 데이터로도 학습이 가능하다.

In [ ]:
#데이터 증강 4
typetools = typetools.new(item_tfms=Resize(128),batch_tfms=aug_transforms(mult=2.5)) # mult 5이상이면 작동 안됨
dls = typetools.dataloaders(path)
#dls.train.show_batch(max_n = 8, nrows=1, unique=True)  #유니크값 동일 이미지에 대한 영역 정보
dls.train.show_batch(max_n = 8, nrows=1, unique=False)

### AI 모델 학습 및 예측

* 자 이제 데이터 전처리를 바탕으로 AI 학습을 해보자
* FastAi는 복잡한 내용을 입력하지 않더라도 간단하게 학습 방법, AI모델 알고리즘, 학습 과정을 보여줄 방법을 설정하면 모델 학습이 가능하다.
* 모델 학습을 진행해보자.
* FastAi 모델 학습에 대한 안내 https://fastai1.fast.ai/vision.learner.html
* learn.fine_tune에 대한 안내 https://docs.fast.ai/callback.schedule.html#Learner.fine_tune

In [ ]:
# 딥러닝 이미지 AI 모델인 cnn중 resnet18을 사용하여 학습할 때마다 학습 오차를 보여주도록 learn 개체를 생성하고
# 학습을 진행한다.
# CNN (컨볼루션 뉴럴 네트워크, 이미지 분류 딥러닝 모델 중 하나)
# CNN을 활용하여 다양한 이미지 데이터용 딥러닝 모델이 만들어졌으며 FastAI에서 제공하는 CNN은
# 다음과 같다
# resnet18, resnet34, resnet50, resnet101, resnet152
# squeezenet1_0, squeezenet1_1
# densenet121, densenet169, densenet201, densenet161
# vgg16_bn, vgg19_bn
# alexnet


learn = vision_learner(dls,resnet18,metrics=error_rate) #이미지 식별 딥러닝 모델의 이름을 입력하기
learn.fine_tune(4)

In [ ]:
#분류 학습의 결과를 시각적으로 보여주는 오차행렬을 생성한 후 보여준다.
interp = ClassificationInterpretation.from_learner(learn) #괄호 안에 모델 이름 집어넣기
interp.plot_confusion_matrix()

In [ ]:
# 값을 다 더하면 그 값의 크기는?
len(dls.valid.items)

In [ ]:
# 학습하는 데 사용한 데이터의 크기는?
len(dls.train.items)

In [ ]:
#가장 오차가 큰 값 20개를 보여줌
interp.plot_top_losses(20, nrows=5) #n개의 값을 n개의 줄에 표시하기

#예측값/실제값/오차/가능성

In [ ]:
#데이터 정리 코드 (선택후 삭제 수정 코드 실행)
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
#위에서 작업 후 반드시 이 코드부터 쭉 4개 실행해주기
cleaner.delete()

In [ ]:
for idx in cleaner.delete() :
  cleaner.fns[idx].unlink()

In [ ]:
cleaner.change()

In [ ]:
for idx, cat in cleaner.change() :
  shutil.move(str(cleaner.fns[idx]), path/cat)

### 모델 사용하기

* 위와 같은 과정을 거쳐서 만든 모델을 실제 사용해봅시다
* AI 모델을 사용하는 이유는 학습을 통해서 만든 모델이 학습하지 않은 상황에서도 잘 적용하기 위해서입니다.
* 만든 모델을 파일로 만들고 그 파일을 불러와 학습하지 않은 이미지를 잘 분류하는지 확인해봅시다.

In [28]:
#학습한 모델의 정보를 파일로 생성하기 확장자 .pkl
learn.export('/content/drive/MyDrive/홍천고 이미지 분류/.pkl')

In [ ]:
#파일을 통해서 모델 불러오기

learn_inf = load_learner('')

In [ ]:
# 모델 정보 확인하기

learn_inf.dls.vocab

### 불러온 모델을 간단한 프로그램으로 만들기

- 파일을 업로드하고
- 그 파일을 보여준 후 이 파일이 어떤 종류인지 예측하는 형태의 간단한 앱을 만들어봅시다.

In [ ]:
btn_upload = widgets.FileUpload(multiple=True)
btn_upload

In [35]:
#불러온 이미지 중 가장 최신(끝에서 첫 번째)을 img에 저장함
img = PILImage.create(btn_upload.data[-1])

In [36]:
# 이미지를 출력하는 기능을 담당하는 out_pl
out_pl = widgets.Output()

In [ ]:
#초기화 후 기능을 이용하여 이미지를 표시함
out_pl.clear_output()
with out_pl : display(img.to_thumb(128,128))
out_pl

In [ ]:
#pred 예측 결과, pred_idx 예측 결과의 위치 probs 가능성
pred, pred_idx, probs = learn_inf.predict(img)

In [ ]:
# 모델 결과값을 출력하는 부분을 생성
lbl_pred = widgets.Label()
lbl_pred.value = f'Predicti on: {pred}, Probability : {probs[pred_idx]:.04f}'
lbl_pred

In [ ]:
#실행 버튼
btn_run = widgets.Button(description = '실행')
btn_run

In [41]:
#위에서 만든 기능을 합쳐서 올라온 이미지를 예측하고 결과를 출력하게 하는 함수 정의 및 버튼에 함수 연결
def on_click_classify(change):
  img = PILImage.create(btn_upload.data[-1])
  out_pl.clear_output()
  with out_pl: display(img.to_thumb(128,128))
  pred,pred_idx,probs=learn_inf.predict(img)
  lbl_pred.value =f'Predction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [ ]:
VBox([widgets.Label('이미지를 분류해보세요!! 그리즐리,판다,북극곰'),btn_upload,btn_run,out_pl,lbl_pred])

In [ ]:
btn_upload = widgets.FileUpload(multiple=True)
btn_upload

In [ ]:


#불러온 이미지 중 가장 최신(끝에서 첫 번째)을 img에 저장함
img = PILImage.create(btn_upload.data[-1])

# 이미지를 출력하는 기능을 담당하는 out_pl
out_pl = widgets.Output()

#pred 예측 결과, pred_idx 예측 결과의 위치 probs 가능성
pred, pred_idx, probs = learn_inf.predict(img)

# 모델 결과값을 출력하는 부분을 생성
lbl_pred = widgets.Label()
lbl_pred.value = f'Predicti on: {pred}, Probability : {probs[pred_idx]:.04f}'
lbl_pred

#실행 버튼
btn_run = widgets.Button(description = '실행')
btn_run

#위에서 만든 기능을 합쳐서 올라온 이미지를 예측하고 결과를 출력하게 하는 함수 정의 및 버튼에 함수 연결
def on_click_classify(change):
  img = PILImage.create(btn_upload.data[-1])
  out_pl.clear_output()
  with out_pl: display(img.to_thumb(128,128))
  pred,pred_idx,probs=learn_inf.predict(img)
  lbl_pred.value =f'Predction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

VBox([widgets.Label('이미지를 분류해보세요!! 그리즐리,판다,북극곰'),btn_upload,btn_run,out_pl,lbl_pred])

